## Notebook to work with interaction dataset merge scenario `DR_CHN_Merging_ZS`
    - Roadway generation
    - Trajdata creation based on reading vehicle tracks from provided csv files

## Changelog
- Friday, Jan 31
    - Creation
    - Toy track generation using way if 10025 from `DR_CHN_Merging_ZS.osm`
- Monday, Feb 3
    - Begin trajdata creation to read in the vehicle tracks. Creation of struct called `INTERACTIONTrajdata`
- Tuesday, Feb 4
    - `Trajdata` reading nominally works. And rendered a scene of course without roadway. Some caveats
        - `car_df_index` line 21 onwards check is commented out
        - No `roadway` involved in the creation of `RecordState`
    - Made a video of replaying the trajdata, of course without roadway, just cars in the blackness
- Wednesday, Feb 5
    - roadway building using `generate_merging_roadway` from `AutonomousMerging.jl`
    - brought in `curevept_overlay` to do overlay the curve points that constitute a road
    - Drove a car in the merging scenario to check that lane semantics works i.e. vehicle understands
    how to keep lane
    - second roadway inspired by William Brannon curved merging lane
- Friday, Feb 7
    - Started pipeline to read Soyeon's centerline data files
    - Generated video replaying interaction dataset on roadways
- Monday, Feb 10
    - Cleanup to prepare for commit to `sisl/AutomotiveInteraction`
- Wednesday, Feb 12
    - Found that the vehicle state needs to have roadway information. Otherwise `observe!` in MOBIL cries
    - This means that we can't initialize the vehicle state the way we have done so far i.e. 
    without providing a `roadway` argument
    - Need to understand `lanes` and `lanetags` and modify roadway creation in terms of how tracks read from
    centerlines txt files into the roadway object
- Friday, Feb 14
    - Got `proj` to work by making `roadway_interaction` in such a way that there are 7 segments, and each segment has only 1 lane. Thus during lane creation, the associated lanetag has `LaneTag(segment_id,1)` as opposed to earlier when the second argument of the LaneTag was going from 1 to 7
- Monday, Feb 17
    - Reorder notebook: make roadway before loading track information
    - function to create the roadway from centerlines called `make_roadway_interaction()` created in
    `roadway_building.jl`
    - function to read in the vehicle tracks called `read_veh_tracks()` created in 
    `../src/veh_track_reading.jl`

In [1]:
# usings
using AutomotiveDrivingModels # Driving simulation platform
using AutoViz # Visualization of roadways and vehicles
using DataFrames # For reading in vehicle tracks into specific type `Trajdata`
using Records # Storing the vehicle tracks in a format that can then be used by `AutomotiveDrivingModels`
using Reel # Make animations of scene visualizations created using `AutoViz`
using DelimitedFiles # Read in centerline txt files into arrays

In [2]:
# Bring functions into notebook: names are self explanatory
include("../src/roadway_building.jl")
include("../src/veh_track_reading.jl")
include("../src/visualization.jl");

## Build the roadway
- functions used (from `AutomotiveInteraction/src/roadway_building.jl)
    - `centerlines_txt2tracks(filename)`, `make_roadway_interaction()`
- Name is `roadway_interaction`
- Read centerlines provided in `../dataset`

In [4]:
roadway_interaction = make_roadway_interaction();

## Read vehicle tracks from csv file
- Functions used (from `AutomotiveInteraction/src/veh_track_reading.jl`)
    - `nframes`, `carsinframe`, `car_df_index`, `read_veh_tracks()`

In [8]:
traj_interaction = read_veh_tracks()

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


Trajdata(3512 frames)

## Driving simulation functions

In [ ]:
# models creation: 2d driver model for the merging roadway
models = Dict{Int64,DriverModel}()
for veh in scene
    models[veh.id] = Tim2DDriver(0.1,mlane=MOBIL(INTERACTION_TIMESTEP,mlon=IntelligentDriverModel()))
end

In [ ]:
# function: get hallucination scenes
"""
    function get_hallucination_scenes
- Hallucinate starting from `start_step` for `nsteps` using `models` and return a list of scenes
- Used by `plot_carwise_pos_vel` to assess position and velocity traces against ground truth

# Returns
- `halluc_scenes_list`: List containing the scenes starting with the ground truth scene at `start_step`

# Examples
```julia
scene_list = get_hallucination_scenes(scene,models=models);
```
"""
function get_hallucination_scenes(scene_halluc;models,start_step=1,duration=5,id_list=[],
        traj=traj_interaction,verbosity = false,timestep=INTERACTION_TIMESTEP,roadway=roadway_interaction)
        # Setting up
    halluc_scenes_list = []
    #scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
    push!(halluc_scenes_list,deepcopy(scene_halluc))
    
    nsteps = duration/timestep
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
        
        if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end
        
        actions = Array{Any}(undef,length(scene_halluc))

            # Propagation of scene forward
        get_actions!(actions,scene_halluc,roadway,models)

        tick!(scene_halluc,roadway,actions,timestep)
        
        push!(halluc_scenes_list,deepcopy(scene_halluc))
    end 
    return halluc_scenes_list
end

In [ ]:
scene_list = get_hallucination_scenes(scene,models=models);

In [ ]:
scenelist2video(scene_list,filename="media/scenelist_to_video.gif")